# 載入套件

In [ ]:
#!pip install tensorflow
#!pip install xgboost

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import tensorflow
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# 載入資料

In [2]:
pd.set_option('display.max_columns', None)
df=pd.read_csv('0416_機器學習.csv')
from sklearn.utils import shuffle
df=shuffle(df)
df

,article_id,sales_channel_id,product_group_name,graphical_appearance_name,colour_group_name,department_name,section_name,garment_group_name,FN,Active,club_member_status,fashion_news_frequency,Year,season,pricelabel,salelabel
39449,562245102,2,Garment Lower body,Denim,Light Blue,Trouser,Womens Everyday Collection,Trousers,0,0,ACTIVE,NONE,2020,Spring,300~399元,一般
76788,770315006,1,Garment Upper body,Solid,White,Jersey,Womens Tailoring,Jersey Fancy,1,1,ACTIVE,REGULARLY,2020,Spring,100~199元,冷門
40519,738943003,2,Garment Upper body,Melange,Greyish Beige,Knitwear,Womens Everyday Collection,Knitwear,0,0,ACTIVE,NONE,2020,Winter,200~299元,一般
26443,688537005,2,Swimwear,Stripe,Dark Blue,Swimwear,"Womens Swimwear, beachwear",Swimwear,1,1,ACTIVE,REGULARLY,2020,Spring,100~199元,一般
20376,794538001,1,Garment Upper body,Solid,White,Knitwear,Womens Everyday Collection,Knitwear,0,0,ACTIVE,NONE,2019,Fall,400~499元,一般
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,796248001,2,Garment Upper body,Melange,Off White,Campaigns,Womens Everyday Collection,Special Offers,0,0,ACTIVE,NONE,2019,Fall,200~299元,冷門
16955,826776001,2,Garment Lower body,Check,Black,Trouser,Womens Everyday Collection,Trousers,1,0,ACTIVE,NONE,2019,Winter,500~599元,一般
84769,547780002,1,Garment Lower body,Melange,Black,Basic 1,Divided Basics,Jersey Basic,0,0,ACTIVE,NONE,2020,Summer,200~299元,熱門
8674,743594003,2,Garment Full body,Solid,White,Dress,Womens Everyday Collection,Dresses Ladies,1,1,ACTIVE,REGULARLY,2019,Summer,300~399元,冷門


# 增加特徵並移除不重要特徵

In [3]:
df=df.drop('department_name',axis=1) #沒刪乾淨 之後確定

In [4]:
df=df.drop('FN',axis=1)

In [5]:
df=df.drop('Active',axis=1)

In [6]:
df=df.drop('club_member_status',axis=1)

In [7]:
df=df.drop('fashion_news_frequency',axis=1)

In [8]:
df

,article_id,sales_channel_id,product_group_name,graphical_appearance_name,colour_group_name,section_name,garment_group_name,Year,season,pricelabel,salelabel
39449,562245102,2,Garment Lower body,Denim,Light Blue,Womens Everyday Collection,Trousers,2020,Spring,300~399元,一般
76788,770315006,1,Garment Upper body,Solid,White,Womens Tailoring,Jersey Fancy,2020,Spring,100~199元,冷門
40519,738943003,2,Garment Upper body,Melange,Greyish Beige,Womens Everyday Collection,Knitwear,2020,Winter,200~299元,一般
26443,688537005,2,Swimwear,Stripe,Dark Blue,"Womens Swimwear, beachwear",Swimwear,2020,Spring,100~199元,一般
20376,794538001,1,Garment Upper body,Solid,White,Womens Everyday Collection,Knitwear,2019,Fall,400~499元,一般
...,...,...,...,...,...,...,...,...,...,...,...
176,796248001,2,Garment Upper body,Melange,Off White,Womens Everyday Collection,Special Offers,2019,Fall,200~299元,冷門
16955,826776001,2,Garment Lower body,Check,Black,Womens Everyday Collection,Trousers,2019,Winter,500~599元,一般
84769,547780002,1,Garment Lower body,Melange,Black,Divided Basics,Jersey Basic,2020,Summer,200~299元,熱門
8674,743594003,2,Garment Full body,Solid,White,Womens Everyday Collection,Dresses Ladies,2019,Summer,300~399元,冷門


# 開始訓練

In [9]:
from sklearn.preprocessing import LabelEncoder #需要特徵都轉數值
labelencoder = LabelEncoder()

df['product_group_name']= labelencoder.fit_transform(df['product_group_name'])
df['graphical_appearance_name']= labelencoder.fit_transform(df['graphical_appearance_name'])
df['colour_group_name']= labelencoder.fit_transform(df['colour_group_name'])
df['section_name']= labelencoder.fit_transform(df['section_name'])
df['garment_group_name']= labelencoder.fit_transform(df['garment_group_name'])
#以下實驗
df['Year']= labelencoder.fit_transform(df['Year'])
df['pricelabel']= labelencoder.fit_transform(df['pricelabel'])
df['season']= labelencoder.fit_transform(df['season'])
df['salelabel']= labelencoder.fit_transform(df['salelabel'])

df.head()

,article_id,sales_channel_id,product_group_name,graphical_appearance_name,colour_group_name,section_name,garment_group_name,Year,season,pricelabel,salelabel
39449,562245102,2,2,5,20,20,15,2,1,3,0
76788,770315006,1,3,19,40,27,5,2,1,1,1
40519,738943003,2,3,12,18,20,6,2,3,2,0
26443,688537005,2,7,20,5,26,14,2,1,1,0
20376,794538001,1,3,19,40,20,6,1,0,4,0


In [10]:
sdf=df

In [11]:
#類型轉換瘦身 使機器學習運算更快
sdf['garment_group_name']=sdf['garment_group_name'].astype('int32')
sdf['section_name']=sdf['section_name'].astype('int32')
sdf['colour_group_name']=sdf['colour_group_name'].astype('int32')
sdf['graphical_appearance_name']=sdf['graphical_appearance_name'].astype('int32')
sdf['product_group_name']=sdf['product_group_name'].astype('int32')
sdf['sales_channel_id']=sdf['sales_channel_id'].astype('int32')
sdf['article_id']=sdf['article_id'].astype('int32')

sdf['Year']=sdf['Year'].astype('int32')
sdf['season']=sdf['season'].astype('int32')
sdf['pricelabel']=sdf['pricelabel'].astype('int32')
sdf['salelabel']=sdf['salelabel'].astype('int32')

sdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95084 entries, 39449 to 36147
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   article_id                 95084 non-null  int32
 1   sales_channel_id           95084 non-null  int32
 2   product_group_name         95084 non-null  int32
 3   graphical_appearance_name  95084 non-null  int32
 4   colour_group_name          95084 non-null  int32
 5   section_name               95084 non-null  int32
 6   garment_group_name         95084 non-null  int32
 7   Year                       95084 non-null  int32
 8   season                     95084 non-null  int32
 9   pricelabel                 95084 non-null  int32
 10  salelabel                  95084 non-null  int32
dtypes: int32(11)
memory usage: 4.7 MB


In [12]:
#設定要預測的Y
labels=sdf['article_id'] 

In [13]:
#移除要預測的Y 表就只剩下X
sdf=sdf.drop('article_id',axis=1) 

In [14]:
#訓練集、測試集分割
import tensorflow as tf 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sdf, labels,test_size=0.2, random_state=80) 

# xgboost

In [15]:
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(        
        n_estimators=8,     #樹的個數
        learning_rate= 0.1,    # 如同學習率 
        max_depth=15,       # 構建樹的深度，越大越容易過擬合    
        subsample=1,         # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0,         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=1,          # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0,        #最大增量步長，我們允許每個樹的權重估計。
        colsample_bytree=1,         # 生成樹時進行的列取樣 
        min_child_weight=1, 
                     # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                     # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                     #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        seed=1000         #隨機種子
#        reg_alpha=0,        # L1 正則項引數
        #scale_pos_weight=1,        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #objective= 'multi:softmax',         #多分類的問題 指定學習任務和相應的學習目標
        #num_class=10,        # 類別數，多分類與 multisoftmax 並用
#        silent=0 ,        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        nthread=4,        # cpu 執行緒數 預設最大
        #eval_metric= 'auc'
)


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
xgbc.fit(X_train, y_train) #模型訓練
XGB_pred=xgbc.predict(X_test) #模型去預測X_test
print(XGB_pred) #此為根據X_test預測出的Y

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[11:25:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[351484026 636207006 565379023 ... 564334002 739953002 811887001]


In [17]:
XGBtrain_score=xgbc.score(X_train, y_train)
print(XGBtrain_score) #X與Y訓練集的準確率

0.8171348942379744


In [18]:
XGBtest_score=xgbc.score(X_test,y_test)
print('The Accuracy of XGBOOST Classifier on testing set:', XGBtest_score) #X與Y測試集的準確率

The Accuracy of XGBOOST Classifier on testing set: 0.79802282168586


In [19]:
# 達到一定標準證明模型準確可以使用
# 只要再製造X_test 就可以print(XGB_pred)得到要的Y

In [20]:
#建立X_test架構====想法
#填入網頁輸入內容連接的函式?

請輸入購買管道=pd.Series('線上') #此為透過網頁輸入的資料
請輸入想要類別=pd.Series('上衣') #此為透過網頁輸入的資料
請輸入想要樣式=pd.Series('圓形') #此為透過網頁輸入的資料

TestX=pd.DataFrame({'sales_channel_id':請輸入購買管道,
                   'product_group_name':請輸入想要類別,
                   'graphical_appearance_name':請輸入想要樣式})
TestX

,sales_channel_id,product_group_name,graphical_appearance_name
0,線上,上衣,圓形


In [21]:
#以上方想法請參考一條龍的檔案

# c4.5

In [30]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [31]:
# Create Decision Tree classifer object
CartC45 = DecisionTreeClassifier(criterion="gini",max_depth=5,min_samples_leaf=20)  #CART-->gini c4.5-->entropy min_samples_leaf=20, max_depth=4

# Train Decision Tree Classifer
CartC45.fit(X_train,y_train)
CartC45_pred =CartC45.predict(X_test)
print(CartC45_pred)

[717490015 720504001 717490001 ... 717490015 717490001 590928022]


In [32]:
print(y_test)

55321    351484026
2126     720504001
55715    565379023
90166    186262001
30801    562252010
           ...    
34482    664405005
95028    554450026
70504    564334002
42502    712216001
4787     811887001
Name: article_id, Length: 19017, dtype: int32


In [33]:
CartC45train_score=CartC45.score(X_train, y_train)
print(CartC45train_score)

0.01733997659957669


In [34]:
CartC45test_score=CartC45.score(X_test,y_test)
print('The Accuracy of CART/C4.5 Classifier on testing set:', CartC45test_score)

The Accuracy of CART/C4.5 Classifier on testing set: 0.013987484881947732


# EXTRA TREE

In [35]:
from sklearn.ensemble import ExtraTreesClassifier
ETreec = ExtraTreesClassifier(max_depth=5,n_estimators=100,criterion='gini')

In [36]:
ETreec.fit(X_train, y_train)
ETree_pred=ETreec.predict(X_test)
print(ETree_pred)

[351484026 720504001 772773001 ... 564334002 712216001 754404004]


In [37]:
ETreetrain_score=ETreec.score(X_train, y_train)
print(ETreetrain_score)

0.5057515085385252


In [38]:
ETreetest_score=ETreec.score(X_test,y_test)
print('The Accuracy of ExtraTrees Classifier on testing set:', ETreetest_score)

The Accuracy of ExtraTrees Classifier on testing set: 0.46090340221906717


# DNN

In [43]:
from sklearn.neural_network import MLPClassifier
DNN = MLPClassifier(hidden_layer_sizes=(5,5,5),  #預設值1層,數目100
         activation='relu',  #{'identity'無，'logistic'為sigmoid，'tanh'，'relu'}，預設='relu'
         solver='adam',   #{'lbfgs'牛頓，'sgd'隨機梯度，'adam'隨機梯度優化}，預設='adam'
         batch_size='auto',  #'auto'為min(200～樣本數)
         learning_rate='adaptive',  #constant固定'，'invscaling隨著時間遞減'，'adaptive誤差減少時不改變'}，預設 ='constant' 
         learning_rate_init=0.001, #優化器為sgd、adam使用
         power_t=0.5,     # 優化器為sgd，則可以設定強化學習率   
         max_iter=1000,    #訓練次數
         shuffle=True,    #隨機設定權重
         random_state=1,   #隨機的基礎值
         momentum=0.9)

In [44]:
# Sckit-Learn
DNN.fit(X_train, y_train)




MLPClassifier(hidden_layer_sizes=(5, 5, 5), learning_rate='adaptive',
              max_iter=1000, random_state=1)

In [45]:
# Sckit-Learn
DNN_pred = DNN.predict(X_test)
DNN_pred

array([351484026, 720504001, 565379023, ..., 564334002, 712216001,
       754404004])

In [46]:
DNNtest_score=DNN.score(X_test,y_test)
print('The Accuracy of DNN Classifier on testing set:', DNNtest_score)

The Accuracy of DNN Classifier on testing set: 0.507493295472472


# 隨機森林

In [40]:
from sklearn.ensemble import RandomForestClassifier

# 建立 Random Forest Classifier 模型
randomForestModel = RandomForestClassifier(n_estimators=100, criterion = 'gini')
# 使用訓練資料訓練模型
randomForestModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = randomForestModel.predict(X_train)

In [42]:
print('訓練集: ',randomForestModel.score(X_train,y_train))
print('測試集: ',randomForestModel.score(X_test,y_test))

訓練集:  0.8500138036204925
測試集:  0.8217384445496135


# catboost

In [23]:
from catboost import CatBoostRegressor

# 建立模型
model02 = CatBoostRegressor(random_state=42,
                         loss_function='RMSE',
                         eval_metric='RMSE',
                         use_best_model=True)
# 使用訓練資料訓練模型
model02.fit(X_train,y_train, eval_set=(X_test, y_test), verbose=0, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
model02_score=model02.score(X_train, y_train)
print(model02_score) #X與Y訓練集的準確率

0.738742776119474


In [25]:
model02_test_score=model02.score(X_test,y_test)
print('The Accuracy of XGBOOST Classifier on testing set:', model02_test_score) #X與Y測試集的準確率

The Accuracy of XGBOOST Classifier on testing set: 0.7349603363886696
